In [1]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
model = AutoModelForVision2Seq.from_pretrained("HuggingFaceTB/SmolVLM-Instruct",
                                                torch_dtype=torch.bfloat16,
                                                _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager").to(DEVICE)


/Users/stephen/Library/Caches/pypoetry/virtualenvs/fine-tuning-oI3l7_A7-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from PIL import Image
from transformers.image_utils import load_image


# Load images
image1 = load_image("https://huggingface.co/spaces/HuggingFaceTB/SmolVLM/resolve/main/example_images/rococo.jpg")
image2 = load_image("https://huggingface.co/spaces/HuggingFaceTB/SmolVLM/resolve/main/example_images/rococo_1.jpg")

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            # {"type": "image"},
            # {"type": "text", "text": "Can you describe the two images?"}
            {"type": "text", "text": "Can you describe the image?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
# inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(DEVICE)


In [3]:
# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])


User:<image>Can you describe the image?
Assistant: The image appears to be a detailed and ornate design, featuring a central circular frame with intricate patterns and embellishments. The frame is surrounded by a symmetrical and highly decorative border, which includes various motifs and elements. The border is composed of a combination of gold and white colors, creating a striking contrast. The central frame contains a painting or illustration, which depicts a scene with multiple figures. The figures are depicted in a classical or mythological style, with elongated bodies, flowing robes, and detailed facial expressions. The background of the painting is filled with various elements, including flowers, trees, and possibly a sky or landscape. The painting is framed by a border of intricate patterns, which adds to the overall aesthetic appeal of the design.

The central frame is circular and features a symmetrical pattern, with a central motif that is repeated in a symmetrical manner. Th